In [41]:
from sklearn.ensemble import RandomForestClassifier
import multiprocessing 
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [42]:
data = pd.read_csv("/home/vrajesh/Downloads/bank/bank.csv", delimiter=";",header='infer')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [28]:
data['poutcome'].value_counts()

unknown    3705
failure     490
other       197
success     129
Name: poutcome, dtype: int64

In [43]:
results = []

def processData(data, outcome_filter):  
    print(outcome_filter)
    df_filter=data[data['poutcome']==outcome_filter]
    df_filter.drop(['poutcome'], axis=1, inplace=True)
    data_new=df_filter
    clf=RandomForestClassifier(n_estimators=100)
    X=data_new.drop('y', axis=1)  # Features
    y=data_new['y']  # Labels

    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    clf.fit(X_train,y_train)
    y_out = clf.predict(X_test)
    X_test['poutcome']=outcome_filter
    cols=list(X_test.columns) + ['Actual']
    out=pd.concat([X_test, y_test], axis=1, ignore_index=True)
    out.columns=cols
    out.reset_index(drop=True, inplace=True)
    final_out=pd.concat([out, pd.DataFrame(y_out)], axis=1, ignore_index=True)
    final_out.columns=list(out.columns)+['Prediction']
    print(final_out.shape)
    return final_out

def collect_results(result):
    results.extend(result.values.tolist())

In [44]:
multiprocessing.cpu_count()

2

In [46]:
if __name__ == "__main__":
    start_time = time.time()  
    
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count()-1)
    for i in data['poutcome'].unique(): 
        pool.apply_async(processData, args=(data.drop(['default','job','marital',
                                         'education','default',
                                         'loan','housing',
                                         'contact','month'], axis=1), i), callback=collect_results)
    pool.close()
    pool.join()
    
    # Converts list of lists to a data frame
    df = pd.DataFrame(results, columns=list(data.drop(['default','job','marital',
                                         'education','default',
                                         'loan','housing',
                                         'contact','month','y'], axis=1).columns) + ['Actual', 'Prediction'])
    print("Dimensions in final test data {}".format(df.shape))
    print("--- %s seconds ---" % (time.time() - start_time))

unknown
(371, 10)
failure
(49, 10)
other
(20, 10)
success
(13, 10)
Dimensions in final test data (906, 10)
--- 2.7560861110687256 seconds ---


In [47]:
df

,age,balance,day,duration,campaign,pdays,previous,poutcome,Actual,Prediction
0,59,92,12,139,2,350,1,failure,no,no
1,37,897,13,119,2,280,2,failure,no,no
2,33,4790,20,137,1,272,2,failure,no,no
3,42,-411,12,78,5,292,3,failure,no,no
4,44,1071,11,143,3,357,1,failure,no,no
...,...,...,...,...,...,...,...,...,...,...
901,46,273,18,910,2,184,4,success,yes,yes
902,47,3676,18,567,1,95,3,success,yes,yes
903,37,0,16,268,2,182,3,success,yes,yes
904,53,94,10,493,1,181,14,success,yes,yes
